In [3]:
pip install rasterio geopandas folium mapclassify

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 22.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import geopandas as gpd
import rasterio
import rasterio.mask
import numpy as np
import folium
from shapely.geometry import mapping
from rasterio.features import geometry_mask
from mapclassify import natural_breaks

ImportError: cannot import name 'natural_breaks' from 'mapclassify' (C:\Users\Karla\anaconda3\envs\geopandas_env\lib\site-packages\mapclassify\__init__.py)

In [ ]:
# Cargar los shapefiles de los departamentos de Perú
base = open(r'../../_data/INEI_LIMITE_DEPARTAMENTAL', 'rb').read()
gdf_departments = gpd.read_file(base)

# Cargar el ráster (asegúrate de que el archivo esté descargado en tu computadora)
raster_path = "builtC.tif"  # Si el archivo no está local, descárgalo desde el enlace proporcionado.
with rasterio.open(raster_path) as src:
    raster_data = src.read(1)
    raster_transform = src.transform
    raster_crs = src.crs

In [ ]:
# Función para extraer porcentaje de área cubierta por cada polígono (departamento)
def extract_area_coverage(shapefile, raster_data, raster_transform):
    results = []
    
    for _, row in shapefile.iterrows():
        # Extraer geometría del polígono
        geometry = [mapping(row['geometry'])]
        
        # Máscara para extraer la parte del ráster que intersecta con el polígono
        out_image, out_transform = rasterio.mask.mask(src, geometry, crop=True)
        
        # Calcular el porcentaje de área cubierta por la zona morfológica
        total_pixels = np.count_nonzero(out_image)
        covered_pixels = np.count_nonzero(out_image > 0)  # Suponiendo que los píxeles cubiertos tienen valor mayor que 0
        
        percentage_covered = (covered_pixels / total_pixels) * 100 if total_pixels > 0 else 0
        results.append(percentage_covered)
    
    shapefile['area_coverage'] = results
    return shapefile

In [ ]:
# Extraer porcentaje de área cubierta por cada departamento
gdf_departments_with_coverage = extract_area_coverage(gdf_departments, raster_data, raster_transform)

# Crear un mapa de coropletas usando Folium
def create_choropleth(gdf, variable, output_html):

# Crear un mapa base
m = folium.Map(location=[-9.19, -75.0152], zoom_start=5)
    
# Añadir el mapa de coropletas
folium.Choropleth(
        geo_data=gdf.__geo_interface__,
        name="choropleth",
        data=gdf,
        columns=["NOMBDEP", variable],
        key_on="feature.properties.NOMBDEP",
        fill_color="YlGnBu",
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=f"% Área Cubierta - {variable}",
    ).add_to(m)

In [ ]:
# Añadir el control de capas
folium.LayerControl().add_to(m)
    
# Guardar el mapa como archivo HTML
m.save(output_html)

# Crear el mapa para el porcentaje de área cubierta
output_html = "departamentos_area_coverage.html"
create_choropleth(gdf_departments_with_coverage, "area_coverage", output_html)

print(f"Mapa guardado en {output_html}")